# LSTM 

## Outline
1. [Imports](#imports)
2. [Data](#data)
3. [Model](#model)
5. [Train](#train)
6. [Predict](#predict)

<a name=imports></a>
## Imports

In [37]:
%load_ext autoreload
%autoreload 2
import os
import datetime

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

import preprocessing

<a name=data></a>
## Data

### Ingestion 

In [38]:
# Check preprocessing.py for implementation
(india_cases_df, india_vacc_df, usa_cases_df, usa_vacc_df) = preprocessing.ingestion()

Parent Directory /covid19-prediction/models
Path: /covid19-prediction/models/../cleaned_datasets/india/daily_cases_india.csv
Exists: True
India Cases:
          Date  Confirmed  Deaths  Recovered
0  2020-01-30        NaN     NaN        NaN
1  2020-01-31        0.0     0.0        0.0
2  2020-02-01        0.0     0.0        0.0
3  2020-02-02        1.0     0.0        0.0
4  2020-02-03        1.0     0.0        0.0 

India Vacc:
          Date  Total_Doses
0  2021-01-15          NaN
1  2021-01-16          NaN
2  2021-01-17      33120.0
3  2021-01-18     229748.0
4  2021-01-19     220786.0 

USA Cases:
          Date  Confirmed  Deaths  Recovered
0  2020-04-12        NaN     NaN        NaN
1  2020-04-13    25255.0  1542.0    11785.0
2  2020-04-14    26694.0  2327.0     6484.0
3  2020-04-15    29318.0  2474.0     6093.0
4  2020-04-16    31235.0  4607.0     5234.0 

USA Vacc:
          Date  Total_Doses
0  2020-12-20     556208.0
1  2020-12-21     614117.0
2  2020-12-22     811071.0
3  2020-

### Pre-Processing

### Multivariate

In [39]:
# Check preprocessing.py for implementation
# Number of Null in India - 563
# Number of Null in USA - 261
(india_multi, usa_multi) = preprocessing.multivariate(india_cases_df, india_vacc_df, usa_cases_df, usa_vacc_df)

India:
          Date  Confirmed  Total_Doses
0  2020-01-30        NaN          NaN
1  2020-01-31        0.0          NaN
2  2020-02-01        0.0          NaN
3  2020-02-02        1.0          NaN
4  2020-02-03        1.0          NaN
USA:
          Date  Confirmed  Total_Doses
0  2020-04-12        NaN          NaN
1  2020-04-13    25255.0          NaN
2  2020-04-14    26694.0          NaN
3  2020-04-15    29318.0          NaN
4  2020-04-16    31235.0          NaN


### Univariate 

In [40]:
# Check preprocessing.py for implementation
# Number of Null in India - 0
# Number of Null in USA - 0
(india_cases_uni, usa_cases_uni) = preprocessing.univariate(india_cases_df, usa_cases_df)

India Cases:
    Confirmed
1        0.0
2        0.0
3        1.0
4        1.0
5        0.0 

USA Cases:
    Confirmed
1    25255.0
2    26694.0
3    29318.0
4    31235.0
5    32175.0 



### Drop Null

In [41]:
(india_multi, usa_multi) = preprocessing.dropNull(india_multi, usa_multi)

India:
            Date  Confirmed  Total_Doses
353  2021-01-17    13788.0      33120.0
354  2021-01-18    10050.0     229748.0
355  2021-01-19    13816.0     220786.0
356  2021-01-20    15244.0     131649.0
357  2021-01-21    14545.0     237050.0
USA:
            Date  Confirmed  Total_Doses
252  2020-12-20   185355.0     556208.0
253  2020-12-21   189716.0     614117.0
254  2020-12-22   201947.0     811071.0
255  2020-12-23   225777.0    1008025.0
256  2020-12-24   204900.0    1476305.0


### Normalize

In [42]:
# Check preprocessing.py for implementation
(india_uni_norm, usa_uni_norm, india_multi_norm, usa_multi_norm) = preprocessing.normalize(india_cases_uni, 
                                                                                           usa_cases_uni, 
                                                                                           india_multi, 
                                                                                           usa_multi)

India Cases univariate:
    Confirmed
1  -0.661747
2  -0.661747
3  -0.661734
4  -0.661734
5  -0.661747 

USA Cases univariate:
    Confirmed
1  -0.596456
2  -0.584433
3  -0.562508
4  -0.546491
5  -0.538637 

India Cases multivariate:
      Confirmed  Total_Doses
353  -0.562592    -1.302360
354  -0.601182    -1.243797
355  -0.562303    -1.246466
356  -0.547561    -1.273014
357  -0.554777    -1.241622 

USA Cases multivariate:
      Confirmed  Total_Doses
252   0.492914    -1.809935
253   0.523316    -1.809749
254   0.608584    -1.809118
255   0.774713    -1.808487
256   0.629171    -1.806987 



### Create Splits

In [43]:
# Create train test splits
(india_uni_train, india_uni_test, usa_uni_train, usa_uni_test) = preprocessing.split(india_uni_norm, usa_uni_norm, 0.2)
(india_multi_train, india_multi_test, 
 usa_multi_train, usa_multi_test) = preprocessing.split(india_multi_norm, usa_multi_norm, 0.2)

India:
      Confirmed
1    -0.661747
2    -0.661747
3    -0.661734
4    -0.661734
5    -0.661747
..         ...
565  -0.199747
566  -0.183686
567  -0.181453
568  -0.209217
569  -0.255301

[569 rows x 1 columns] 

USA:
      Confirmed
1    -0.596456
2    -0.584433
3    -0.562508
4    -0.546491
5    -0.538637
..         ...
507   0.882933
508   0.655423
509   1.161010
510  -0.188293
511  -0.412010

[511 rows x 1 columns] 

India:
      Confirmed  Total_Doses
353  -0.562592    -1.302360
354  -0.601182    -1.243797
355  -0.562303    -1.246466
356  -0.547561    -1.273014
357  -0.554777    -1.241622
..         ...          ...
638  -0.557172     1.442780
639  -0.572482     0.707335
640  -0.575745    -0.078616
641  -0.597332     0.183795
642  -0.582053    -0.697398

[279 rows x 2 columns] 

USA:
      Confirmed  Total_Doses
252   0.492914    -1.809935
253   0.523316    -1.809749
254   0.608584    -1.809118
255   0.774713    -1.808487
256   0.629171    -1.806987
..         ...          ...
55

### Data Transformation
Simple conversion from pandas dataframe to numpy array by extracting the values

In [44]:
# Univariate
india_uni_train_np, india_uni_test_np = india_uni_train.values,  india_uni_test.values
usa_uni_train_np, usa_uni_test_np = usa_uni_train.values, usa_uni_test.values
# Multivariate
india_multi_train_np, india_multi_test_np = india_multi_train.values,  india_multi_test.values
usa_multi_train_np, usa_multi_test_np = usa_multi_train.values, usa_multi_test.values

In [45]:
print(india_uni_train_np.shape, india_uni_test_np.shape)
print(india_multi_train_np.shape, india_multi_test_np.shape)

(569, 1) (143, 1)
(279, 2) (70, 2)


### Reshape
Converts the exiting train data set to match the input parameters of an LSTM (Samples, Time Steps, Features). The `lstm_data_transform()` method takes in 3 arguments: X data, y data and number of features.

We will be performing autoregression on the univariate data, therefore the X and y values are the same. As for the multivariate data, we will be using the confirmed column (index 0) as the y value.

In [46]:
# Univariate
india_uni_train_X, india_uni_train_y = preprocessing.lstm_data_transform(india_uni_train_np, india_uni_train_np, 3)
usa_uni_train_X, usa_uni_train_y = preprocessing.lstm_data_transform(usa_uni_train_np, usa_uni_train_np, 3)
# Multivariate
india_multi_train_X, india_multi_train_y = preprocessing.lstm_data_transform(india_multi_train_np, india_multi_train_np[:,0], 3)
usa_multi_train_X, usa_multi_train_y = preprocessing.lstm_data_transform(usa_multi_train_np, usa_multi_train_np[:,0], 3)

In [47]:
print("Univariate India shape:",india_uni_train_X.shape, india_uni_train_y.shape)
print("Multivariate India shape:",india_multi_train_X.shape, india_multi_train_y.shape)

Univariate India shape: (566, 3, 1) (566, 1)
Multivariate India shape: (276, 3, 2) (276,)


The univariate array shape for the India dataset has 566 samples, 3 as the time step, and 1 feature for the X values. The y values is just the 566 samples.

Similarly, the multivariate array shape for the India dataset has 276 samples, 3 as the time step, and 2 features for the X values. The y values is just the 276 samples of the 0th column (Confirmed cases).

<a name=model></a>
## Model
We will define 4 models in total. One for the univariate data and another for the multivariate for each country.

In [48]:
import LSTMmodels
from keras.wrappers.scikit_learn import KerasRegressor
import tensorflow as tf

In [79]:
univariate_model_ind = LSTMmodels.build_lstm(time_steps=india_uni_train_X.shape[1], features=india_uni_train_X.shape[2])
univariate_model_usa = LSTMmodels.build_lstm(time_steps=usa_uni_train_X.shape[1], features=usa_uni_train_X.shape[2])

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 100)               40800     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 101       
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 100)               40800     
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 101       
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________


In [80]:
multivariate_model_ind = LSTMmodels.build_lstm(time_steps=india_multi_train_X.shape[1], features=india_multi_train_X.shape[2])
multivariate_model_usa = LSTMmodels.build_lstm(time_steps=usa_multi_train_X.shape[1], features=usa_multi_train_X.shape[2])

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 100)               41200     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 101       
Total params: 41,301
Trainable params: 41,301
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 100)               41200     
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 101       
Total params: 41,301
Trainable params: 41,301
Non-trainable params: 0
_________________________________________________________________


<a name=train></a>
## Train 

In [51]:
# Get the path of the current working directory
curPath = os.getcwd()
# Appened the parent directory to the current path to step out of the current folder
parentDir = os.path.abspath(os.path.join(curPath, os.pardir))
# Tensorboard callback
logdir = os.path.join(parentDir+"/logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [52]:
def train_models(model,train_X,train_y,test_X,test_y):
    model.fit(train_X,train_y,
              validation_data=(test_X,test_y),
              epochs=300,
              callbacks=[tensorboard_callback])
    return model
#     model.save(save_name)

In [81]:
# Train the model on univariate India dataset 
univariate_model_ind = LSTMmodels.train_model(univariate_model_ind,india_uni_train_X,india_uni_train_y,tensorboard_callback)

Epoch 1/300
18/18 [==============================] - 1s 18ms/step - loss: 0.5186 - mae: 0.5186
Epoch 2/300
18/18 [==============================] - 0s 5ms/step - loss: 0.3180 - mae: 0.3180
Epoch 3/300
18/18 [==============================] - 0s 5ms/step - loss: 0.1151 - mae: 0.1151
Epoch 4/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0969 - mae: 0.0969
Epoch 5/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0932 - mae: 0.0932
Epoch 6/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0903 - mae: 0.0903
Epoch 7/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0937 - mae: 0.0937
Epoch 8/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0899 - mae: 0.0899
Epoch 9/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0868 - mae: 0.0868
Epoch 10/300
18/18 [==============================] - 0s 4ms/step - loss: 0.0836 - mae: 0.0836
Epoch 11/300
18/18 [==============================] - 0s 5

18/18 [==============================] - 0s 4ms/step - loss: 0.0666 - mae: 0.0666
Epoch 88/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0705 - mae: 0.0705
Epoch 89/300
18/18 [==============================] - 0s 4ms/step - loss: 0.0711 - mae: 0.0711
Epoch 90/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0702 - mae: 0.0702
Epoch 91/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0697 - mae: 0.0697
Epoch 92/300
18/18 [==============================] - 0s 4ms/step - loss: 0.0662 - mae: 0.0662
Epoch 93/300
18/18 [==============================] - 0s 4ms/step - loss: 0.0664 - mae: 0.0664
Epoch 94/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0668 - mae: 0.0668
Epoch 95/300
18/18 [==============================] - 0s 4ms/step - loss: 0.0695 - mae: 0.0695
Epoch 96/300
18/18 [==============================] - 0s 4ms/step - loss: 0.0706 - mae: 0.0706
Epoch 97/300
18/18 [==============================] - 0s 5ms/st

18/18 [==============================] - 0s 5ms/step - loss: 0.0624 - mae: 0.0624
Epoch 173/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0627 - mae: 0.0627
Epoch 174/300
18/18 [==============================] - 0s 4ms/step - loss: 0.0614 - mae: 0.0614
Epoch 175/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0651 - mae: 0.0651
Epoch 176/300
18/18 [==============================] - 0s 4ms/step - loss: 0.0657 - mae: 0.0657
Epoch 177/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0637 - mae: 0.0637
Epoch 178/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0633 - mae: 0.0633
Epoch 179/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0634 - mae: 0.0634
Epoch 180/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0628 - mae: 0.0628
Epoch 181/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0620 - mae: 0.0620
Epoch 182/300
18/18 [==============================] -

18/18 [==============================] - 0s 4ms/step - loss: 0.0548 - mae: 0.0548
Epoch 258/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0562 - mae: 0.0562
Epoch 259/300
18/18 [==============================] - 0s 4ms/step - loss: 0.0557 - mae: 0.0557
Epoch 260/300
18/18 [==============================] - 0s 4ms/step - loss: 0.0541 - mae: 0.0541
Epoch 261/300
18/18 [==============================] - 0s 4ms/step - loss: 0.0543 - mae: 0.0543
Epoch 262/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0549 - mae: 0.0549
Epoch 263/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0527 - mae: 0.0527
Epoch 264/300
18/18 [==============================] - 0s 4ms/step - loss: 0.0536 - mae: 0.0536
Epoch 265/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0550 - mae: 0.0550
Epoch 266/300
18/18 [==============================] - 0s 5ms/step - loss: 0.0539 - mae: 0.0539
Epoch 267/300
18/18 [==============================] -

In [ ]:
# Train the model on univariate USA dataset 
univariate_model_usa = LSTMmodels.train_model(univariate_model_usa,usa_uni_train_X,usa_uni_train_y,tensorboard_callback)

In [ ]:
# Train the model on multivariate India dataset
multivariate_model_ind = LSTMmodels.train_model(multivariate_model_ind,india_multi_train_X,india_multi_train_y,tensorboard_callback)

In [ ]:
# Train the model on multivariate USA dataset
multivariate_model_usa = LSTMmodels.train_model(multivariate_model_usa,usa_multi_train_X,usa_multi_train_y,tensorboard_callback)

<a name=predict></a>
## Predict

In [82]:
import evaluate

In [129]:
timesteps = 3

In [84]:
india_uni_test_np.shape

(143, 1)

In [124]:
yhat_uni_india = evaluate.make_predictions(univariate_model_ind,india_uni_test_np,timesteps,india_uni_test_np.shape[1])

hello
itr size: 140
X_data shape (1, 3, 1)
X_data: [[[-0.33247168]
  [-0.32728406]
  [-0.16803078]]]
X_data input: [[[-0.33247168]
  [-0.32728406]
  [-0.16803078]]]
forecast: [[[-0.19237554]]]
In loop: [[[-0.33247168]
  [-0.32728406]
  [-0.16803078]
  [-0.19237554]]]
forecast: [[[-0.20443258]]]
In loop: [[[-0.33247168]
  [-0.32728406]
  [-0.16803078]
  [-0.19237554]
  [-0.20443258]]]
forecast: [[[-0.21015975]]]
In loop: [[[-0.33247168]
  [-0.32728406]
  [-0.16803078]
  [-0.19237554]
  [-0.20443258]
  [-0.21015975]]]
forecast: [[[-0.21527316]]]
In loop: [[[-0.33247168]
  [-0.32728406]
  [-0.16803078]
  [-0.19237554]
  [-0.20443258]
  [-0.21015975]
  [-0.21527316]]]
forecast: [[[-0.22012827]]]
In loop: [[[-0.33247168]
  [-0.32728406]
  [-0.16803078]
  [-0.19237554]
  [-0.20443258]
  [-0.21015975]
  [-0.21527316]
  [-0.22012827]]]
forecast: [[[-0.22411342]]]
In loop: [[[-0.33247168]
  [-0.32728406]
  [-0.16803078]
  [-0.19237554]
  [-0.20443258]
  [-0.21015975]
  [-0.21527316]
  [-0.22012

forecast: [[[-0.2537527]]]
In loop: [[[-0.33247168]
  [-0.32728406]
  [-0.16803078]
  [-0.19237554]
  [-0.20443258]
  [-0.21015975]
  [-0.21527316]
  [-0.22012827]
  [-0.22411342]
  [-0.22748199]
  [-0.23050056]
  [-0.23304658]
  [-0.23534735]
  [-0.2373812 ]
  [-0.23921314]
  [-0.24087313]
  [-0.24234003]
  [-0.24363592]
  [-0.24490489]
  [-0.24596263]
  [-0.24700244]
  [-0.24788688]
  [-0.24876979]
  [-0.24953128]
  [-0.25021982]
  [-0.25088555]
  [-0.25154763]
  [-0.2519846 ]
  [-0.25244796]
  [-0.25287521]
  [-0.25331005]
  [-0.25375271]]]
forecast: [[[-0.25395527]]]
In loop: [[[-0.33247168]
  [-0.32728406]
  [-0.16803078]
  [-0.19237554]
  [-0.20443258]
  [-0.21015975]
  [-0.21527316]
  [-0.22012827]
  [-0.22411342]
  [-0.22748199]
  [-0.23050056]
  [-0.23304658]
  [-0.23534735]
  [-0.2373812 ]
  [-0.23921314]
  [-0.24087313]
  [-0.24234003]
  [-0.24363592]
  [-0.24490489]
  [-0.24596263]
  [-0.24700244]
  [-0.24788688]
  [-0.24876979]
  [-0.24953128]
  [-0.25021982]
  [-0.2508855

forecast: [[[-0.25766432]]]
In loop: [[[-0.33247168]
  [-0.32728406]
  [-0.16803078]
  [-0.19237554]
  [-0.20443258]
  [-0.21015975]
  [-0.21527316]
  [-0.22012827]
  [-0.22411342]
  [-0.22748199]
  [-0.23050056]
  [-0.23304658]
  [-0.23534735]
  [-0.2373812 ]
  [-0.23921314]
  [-0.24087313]
  [-0.24234003]
  [-0.24363592]
  [-0.24490489]
  [-0.24596263]
  [-0.24700244]
  [-0.24788688]
  [-0.24876979]
  [-0.24953128]
  [-0.25021982]
  [-0.25088555]
  [-0.25154763]
  [-0.2519846 ]
  [-0.25244796]
  [-0.25287521]
  [-0.25331005]
  [-0.25375271]
  [-0.25395527]
  [-0.25419858]
  [-0.25446004]
  [-0.25466296]
  [-0.25488499]
  [-0.25511563]
  [-0.25535268]
  [-0.25558272]
  [-0.25579524]
  [-0.25605676]
  [-0.25625896]
  [-0.25650936]
  [-0.25673875]
  [-0.25696474]
  [-0.25719592]
  [-0.25720188]
  [-0.25725991]
  [-0.25748861]
  [-0.25762826]
  [-0.25766432]]]
forecast: [[[-0.25768515]]]
In loop: [[[-0.33247168]
  [-0.32728406]
  [-0.16803078]
  [-0.19237554]
  [-0.20443258]
  [-0.210159

forecast: [[[-0.25768515]]]
In loop: [[[-0.33247168]
  [-0.32728406]
  [-0.16803078]
  [-0.19237554]
  [-0.20443258]
  [-0.21015975]
  [-0.21527316]
  [-0.22012827]
  [-0.22411342]
  [-0.22748199]
  [-0.23050056]
  [-0.23304658]
  [-0.23534735]
  [-0.2373812 ]
  [-0.23921314]
  [-0.24087313]
  [-0.24234003]
  [-0.24363592]
  [-0.24490489]
  [-0.24596263]
  [-0.24700244]
  [-0.24788688]
  [-0.24876979]
  [-0.24953128]
  [-0.25021982]
  [-0.25088555]
  [-0.25154763]
  [-0.2519846 ]
  [-0.25244796]
  [-0.25287521]
  [-0.25331005]
  [-0.25375271]
  [-0.25395527]
  [-0.25419858]
  [-0.25446004]
  [-0.25466296]
  [-0.25488499]
  [-0.25511563]
  [-0.25535268]
  [-0.25558272]
  [-0.25579524]
  [-0.25605676]
  [-0.25625896]
  [-0.25650936]
  [-0.25673875]
  [-0.25696474]
  [-0.25719592]
  [-0.25720188]
  [-0.25725991]
  [-0.25748861]
  [-0.25762826]
  [-0.25766432]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [

forecast: [[[-0.25768515]]]
In loop: [[[-0.33247168]
  [-0.32728406]
  [-0.16803078]
  [-0.19237554]
  [-0.20443258]
  [-0.21015975]
  [-0.21527316]
  [-0.22012827]
  [-0.22411342]
  [-0.22748199]
  [-0.23050056]
  [-0.23304658]
  [-0.23534735]
  [-0.2373812 ]
  [-0.23921314]
  [-0.24087313]
  [-0.24234003]
  [-0.24363592]
  [-0.24490489]
  [-0.24596263]
  [-0.24700244]
  [-0.24788688]
  [-0.24876979]
  [-0.24953128]
  [-0.25021982]
  [-0.25088555]
  [-0.25154763]
  [-0.2519846 ]
  [-0.25244796]
  [-0.25287521]
  [-0.25331005]
  [-0.25375271]
  [-0.25395527]
  [-0.25419858]
  [-0.25446004]
  [-0.25466296]
  [-0.25488499]
  [-0.25511563]
  [-0.25535268]
  [-0.25558272]
  [-0.25579524]
  [-0.25605676]
  [-0.25625896]
  [-0.25650936]
  [-0.25673875]
  [-0.25696474]
  [-0.25719592]
  [-0.25720188]
  [-0.25725991]
  [-0.25748861]
  [-0.25762826]
  [-0.25766432]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [

...

forecast: [[[-0.25768515]]]
In loop: [[[-0.33247168]
  [-0.32728406]
  [-0.16803078]
  [-0.19237554]
  [-0.20443258]
  [-0.21015975]
  [-0.21527316]
  [-0.22012827]
  [-0.22411342]
  [-0.22748199]
  [-0.23050056]
  [-0.23304658]
  [-0.23534735]
  [-0.2373812 ]
  [-0.23921314]
  [-0.24087313]
  [-0.24234003]
  [-0.24363592]
  [-0.24490489]
  [-0.24596263]
  [-0.24700244]
  [-0.24788688]
  [-0.24876979]
  [-0.24953128]
  [-0.25021982]
  [-0.25088555]
  [-0.25154763]
  [-0.2519846 ]
  [-0.25244796]
  [-0.25287521]
  [-0.25331005]
  [-0.25375271]
  [-0.25395527]
  [-0.25419858]
  [-0.25446004]
  [-0.25466296]
  [-0.25488499]
  [-0.25511563]
  [-0.25535268]
  [-0.25558272]
  [-0.25579524]
  [-0.25605676]
  [-0.25625896]
  [-0.25650936]
  [-0.25673875]
  [-0.25696474]
  [-0.25719592]
  [-0.25720188]
  [-0.25725991]
  [-0.25748861]
  [-0.25762826]
  [-0.25766432]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [

forecast: [[[-0.25768515]]]
In loop: [[[-0.33247168]
  [-0.32728406]
  [-0.16803078]
  [-0.19237554]
  [-0.20443258]
  [-0.21015975]
  [-0.21527316]
  [-0.22012827]
  [-0.22411342]
  [-0.22748199]
  [-0.23050056]
  [-0.23304658]
  [-0.23534735]
  [-0.2373812 ]
  [-0.23921314]
  [-0.24087313]
  [-0.24234003]
  [-0.24363592]
  [-0.24490489]
  [-0.24596263]
  [-0.24700244]
  [-0.24788688]
  [-0.24876979]
  [-0.24953128]
  [-0.25021982]
  [-0.25088555]
  [-0.25154763]
  [-0.2519846 ]
  [-0.25244796]
  [-0.25287521]
  [-0.25331005]
  [-0.25375271]
  [-0.25395527]
  [-0.25419858]
  [-0.25446004]
  [-0.25466296]
  [-0.25488499]
  [-0.25511563]
  [-0.25535268]
  [-0.25558272]
  [-0.25579524]
  [-0.25605676]
  [-0.25625896]
  [-0.25650936]
  [-0.25673875]
  [-0.25696474]
  [-0.25719592]
  [-0.25720188]
  [-0.25725991]
  [-0.25748861]
  [-0.25762826]
  [-0.25766432]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [

forecast: [[[-0.25768515]]]
In loop: [[[-0.33247168]
  [-0.32728406]
  [-0.16803078]
  [-0.19237554]
  [-0.20443258]
  [-0.21015975]
  [-0.21527316]
  [-0.22012827]
  [-0.22411342]
  [-0.22748199]
  [-0.23050056]
  [-0.23304658]
  [-0.23534735]
  [-0.2373812 ]
  [-0.23921314]
  [-0.24087313]
  [-0.24234003]
  [-0.24363592]
  [-0.24490489]
  [-0.24596263]
  [-0.24700244]
  [-0.24788688]
  [-0.24876979]
  [-0.24953128]
  [-0.25021982]
  [-0.25088555]
  [-0.25154763]
  [-0.2519846 ]
  [-0.25244796]
  [-0.25287521]
  [-0.25331005]
  [-0.25375271]
  [-0.25395527]
  [-0.25419858]
  [-0.25446004]
  [-0.25466296]
  [-0.25488499]
  [-0.25511563]
  [-0.25535268]
  [-0.25558272]
  [-0.25579524]
  [-0.25605676]
  [-0.25625896]
  [-0.25650936]
  [-0.25673875]
  [-0.25696474]
  [-0.25719592]
  [-0.25720188]
  [-0.25725991]
  [-0.25748861]
  [-0.25762826]
  [-0.25766432]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [

forecast: [[[-0.25768515]]]
In loop: [[[-0.33247168]
  [-0.32728406]
  [-0.16803078]
  [-0.19237554]
  [-0.20443258]
  [-0.21015975]
  [-0.21527316]
  [-0.22012827]
  [-0.22411342]
  [-0.22748199]
  [-0.23050056]
  [-0.23304658]
  [-0.23534735]
  [-0.2373812 ]
  [-0.23921314]
  [-0.24087313]
  [-0.24234003]
  [-0.24363592]
  [-0.24490489]
  [-0.24596263]
  [-0.24700244]
  [-0.24788688]
  [-0.24876979]
  [-0.24953128]
  [-0.25021982]
  [-0.25088555]
  [-0.25154763]
  [-0.2519846 ]
  [-0.25244796]
  [-0.25287521]
  [-0.25331005]
  [-0.25375271]
  [-0.25395527]
  [-0.25419858]
  [-0.25446004]
  [-0.25466296]
  [-0.25488499]
  [-0.25511563]
  [-0.25535268]
  [-0.25558272]
  [-0.25579524]
  [-0.25605676]
  [-0.25625896]
  [-0.25650936]
  [-0.25673875]
  [-0.25696474]
  [-0.25719592]
  [-0.25720188]
  [-0.25725991]
  [-0.25748861]
  [-0.25762826]
  [-0.25766432]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [

In loop: [[[-0.33247168]
  [-0.32728406]
  [-0.16803078]
  [-0.19237554]
  [-0.20443258]
  [-0.21015975]
  [-0.21527316]
  [-0.22012827]
  [-0.22411342]
  [-0.22748199]
  [-0.23050056]
  [-0.23304658]
  [-0.23534735]
  [-0.2373812 ]
  [-0.23921314]
  [-0.24087313]
  [-0.24234003]
  [-0.24363592]
  [-0.24490489]
  [-0.24596263]
  [-0.24700244]
  [-0.24788688]
  [-0.24876979]
  [-0.24953128]
  [-0.25021982]
  [-0.25088555]
  [-0.25154763]
  [-0.2519846 ]
  [-0.25244796]
  [-0.25287521]
  [-0.25331005]
  [-0.25375271]
  [-0.25395527]
  [-0.25419858]
  [-0.25446004]
  [-0.25466296]
  [-0.25488499]
  [-0.25511563]
  [-0.25535268]
  [-0.25558272]
  [-0.25579524]
  [-0.25605676]
  [-0.25625896]
  [-0.25650936]
  [-0.25673875]
  [-0.25696474]
  [-0.25719592]
  [-0.25720188]
  [-0.25725991]
  [-0.25748861]
  [-0.25762826]
  [-0.25766432]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]
  [-0.25768515]

In [125]:
yhat_uni_india.shape

(1, 143, 1)

## Evaluate

In [127]:
from tensorflow.keras.losses import MeanAbsolutePercentageError, MeanAbsoluteError

### Univariate Non Stacked -- Rolling

#### India

In [128]:
mape = MeanAbsolutePercentageError()
mape(india_uni_test_np.flatten()[timesteps:], yhat_uni_india.flatten()[timestep:]).numpy()

NameError: name 'timestep' is not defined

In [ ]:
mae = MeanAbsoluteError()
mae(india_cases_test_y, yhat_uni_non_stacked_india).numpy()

#### USA

In [ ]:
mape = MeanAbsolutePercentageError()
mape(usa_cases_test_y, yhat_uni_non_stacked_usa).numpy()

In [ ]:
mae = MeanAbsoluteError()
mae(usa_cases_test_y, yhat_uni_non_stacked_usa).numpy()

### Univariate Non Stacked -- Long-Term

#### India 

In [ ]:
mape = MeanAbsolutePercentageError()
mape(india_cases_test_y, yhat_uni_non_stacked_long_india).numpy()

In [ ]:
mae = MeanAbsoluteError()
mae(india_cases_test_y, yhat_uni_non_stacked_long_india).numpy()

#### USA 

In [ ]:
mape = MeanAbsolutePercentageError()
mape(usa_cases_test_y, yhat_uni_non_stacked_long_usa).numpy()

In [ ]:
mae = MeanAbsoluteError()
mae(usa_cases_test_y, yhat_uni_non_stacked_long_usa).numpy()

### Univariate Stacked -- Rolling

#### India 

In [ ]:
mape = MeanAbsolutePercentageError()
mape(india_cases_test_y, yhat_uni_stacked_india).numpy()

In [ ]:
mae = MeanAbsoluteError()
mae(india_cases_test_y, yhat_uni_stacked_india).numpy()

#### USA 

In [ ]:
mape = MeanAbsolutePercentageError()
mape(usa_cases_test_y, yhat_uni_stacked_usa).numpy()

In [ ]:
mae = MeanAbsoluteError()
mae(usa_cases_test_y, yhat_uni_stacked_usa).numpy()

### Univariate Stacked -- Long-Term

#### India 

In [ ]:
mape = MeanAbsolutePercentageError()
mape(india_cases_test_y, yhat_uni_stacked_long_india).numpy()

In [ ]:
mae = MeanAbsoluteError()
mae(india_cases_test_y, yhat_uni_stacked_long_india).numpy()

#### USA

In [ ]:
mape = MeanAbsolutePercentageError()
mape(usa_cases_test_y, yhat_uni_stacked_long_usa).numpy()

In [ ]:
mae = MeanAbsoluteError()
mae(usa_cases_test_y, yhat_uni_stacked_long_usa).numpy()

## Plots

In [ ]:
# Reversing Z-score normalization

casted_mean = india_cases_mean.to_numpy()
casted_std = india_cases_std.to_numpy()

india_cases_test_scaled = (india_cases_std[0]*india_cases_test)+india_cases_mean[0]
yhat_uni_non_stacked_india_scaled = (india_cases_std[0]*yhat_uni_non_stacked_india)+india_cases_mean[0]
yhat_uni_non_stacked_long_india_scaled = (india_cases_std[0]*yhat_uni_non_stacked_long_india)+india_cases_mean[0]
yhat_uni_stacked_india_scaled = (india_cases_std[0]*yhat_uni_stacked_india)+india_cases_mean[0]
yhat_uni_stacked_long_india_scaled = (india_cases_std[0]*yhat_uni_stacked_long_india)+india_cases_mean[0]

usa_cases_test_scaled = (usa_cases_std[0]*usa_cases_test)+usa_cases_mean[0]
yhat_uni_non_stacked_usa_scaled = (usa_cases_std[0]*yhat_uni_non_stacked_usa)+usa_cases_mean[0]
yhat_uni_non_stacked_long_usa_scaled = (usa_cases_std[0]*yhat_uni_non_stacked_long_usa)+usa_cases_mean[0]
yhat_uni_stacked_usa_scaled = (usa_cases_std[0]*yhat_uni_stacked_usa)+usa_cases_mean[0]
yhat_uni_stacked_long_usa_scaled = (usa_cases_std[0]*yhat_uni_stacked_long_usa)+usa_cases_mean[0]

print(yhat_uni_non_stacked_india_scaled.size)
print(india_cases_test_scaled.size)

In [ ]:
def plot_fore_test(test, fore, title):
    
    fig, ax = plt.subplots()
    fig.set_size_inches(12, 8)

    ax.plot(test, color='blue', label='Test')
    ax.plot(fore, color='red', label='Forecast')
    ax.legend(loc='best')
    plt.title(title)
    plt.show()

### Univariate Non Stacked -- Rolling

In [ ]:
plot_fore_test(india_cases_test_scaled, yhat_uni_non_stacked_india_scaled, title='Daily cases')

In [ ]:
plot_fore_test(usa_cases_test_scaled, yhat_uni_non_stacked_usa_scaled, title='Daily cases')

### Univariate Non Stacked -- Long-Term

In [ ]:
plot_fore_test(india_cases_test_scaled, yhat_uni_non_stacked_long_india_scaled, title='Daily cases')

In [ ]:
plot_fore_test(usa_cases_test_scaled, yhat_uni_non_stacked_long_usa_scaled, title='Daily cases')

### Univariate Stacked -- Rolling

In [ ]:
plot_fore_test(india_cases_test_scaled, yhat_uni_stacked_india_scaled, title='Daily cases')

In [ ]:
plot_fore_test(usa_cases_test_scaled, yhat_uni_stacked_usa_scaled, title='Daily cases')

### Univariate Stacked -- Long-Term

In [ ]:
plot_fore_test(india_cases_test_scaled, yhat_uni_stacked_long_india_scaled, title='Daily cases')

In [ ]:
plot_fore_test(usa_cases_test_scaled, yhat_uni_stacked_long_usa_scaled, title='Daily cases')

#  Evaluation Metrics

In [ ]:
keras_mape = MeanAbsolutePercentageError()
keras_mape(india_cases_test_scaled[3:], yhat_uni_non_stacked_india_scaled).numpy()

In [ ]:
def MAPE(Y_actual,Y_Predicted, title):
    mask = Y_actual != 0
    
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual)[mask])*100
    print(f"MAPE of {title} is {mape}%")
    
mape_non_stacked_india = MAPE(india_cases_test_scaled[3:], yhat_uni_non_stacked_india_scaled, title="India Non-stacked")
mape_stacked_india = MAPE(india_cases_test_scaled[3:], yhat_uni_stacked_india_scaled, title="India Stacked")
# mape_vacc = MAPE(val['Confirmed'], roll_fc, title="Daily Cases")

In [ ]:
mape_non_stacked_usa = MAPE(usa_cases_test_scaled[3:], yhat_uni_non_stacked_usa_scaled, title="USA Non-stacked")
mape_stacked_usa = MAPE(usa_cases_test_scaled[3:], yhat_uni_stacked_usa_scaled, title="USA Stacked")

In [ ]:
from sklearn.metrics import mean_absolute_error

print('MAE of USA Non-stacked:', mean_absolute_error(usa_cases_test_scaled[3:], yhat_uni_non_stacked_usa_scaled))
print('MAE of USA Stacked:', mean_absolute_error(usa_cases_test_scaled[3:], yhat_uni_stacked_usa_scaled))

In [ ]:
print('MAE of India Non-stacked:', mean_absolute_error(india_cases_test_scaled[3:], yhat_uni_non_stacked_india_scaled))
print('MAE of Inida stacked:', mean_absolute_error(india_cases_test_scaled[3:], yhat_uni_stacked_india_scaled))